In [269]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

In [270]:
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
         yield start_date + datetime.timedelta(i)

In [271]:
dataframe = pd.read_csv('../normalizados/dados/3500105.csv')

z = []
X = []

dias = 0

for x in range(0, 148 - dias):
    X.append(dataframe.loc[x][['media_movel_casos', 'media_movel_obitos', 'ocupacao_leitos','media_isolamento', 'Mean.R']])
    
for x in range(dias, 148):
    z.append(dataframe.loc[x]['diagnostico_covid19'])
    


In [272]:
# Selecionando X e y do dataframe

#print(dataframe.columns)

#X = dataframe[['media_movel_casos', 'media_movel_obitos', 'ocupacao_leitos','media_isolamento', 'Mean.R']]
selector = VarianceThreshold()
X = selector.fit_transform(X)
pca = PCA(n_components=4)
X = pca.fit_transform(X)
y = z

In [273]:
reg = LinearRegression()
scores = cross_val_score(reg, X, y, cv=5)

In [274]:
print(scores)

[0.39053662 0.33052697 0.27774548 0.0991976  0.34079436]


In [275]:
print(scores.mean()*100)

28.776020632895637
